# Processing Envisat Data with stripmapApp.py

**Author**: Eric Fielding, based on stripmapApp notebook by Heresh Fattahi, Piyush Agram

In this notebook, we will walk through the various steps of processing an Envisat pair with **stripmapApp.py**. 

stripmapApp.py is a pair-by-pair interferometric processor that takes as input two SAR acquisitions acquired in stripmap mode. stripmapApp.py will not work for other acquisition mode such as TOPS or ScanSAR. At this time, ISCE's topsApp supports TOPS mode acquisitions in SLC data format from Sentinel-1 A and B. Processing of stripmap data is supported from RAW or SLC level data inputs.

In this example we will use an ESA Envisat pair capturing the surface deformation that resulted from the April 2010 Mw 7.2 El Mayor-Cucapah Earthquake in Baja California, Mexico.

See the `stripmapApp.ipynb` for the full explanation of the methods used.

## Envisat data

The European Space Agency (ESA) built, launched and operated the Envisat satellite from 2003 to 2012. They lowered the orbit in October 2010 as they were getting low on fuel and stopped controlling the orbit inclination, so data after that time is generally not useful for InSAR. The new orbit had a 30-day repeat, shorter than the original 35-day repeat. In this notebook, we will process the Envisat raw (level 0) data that is available to WInSAR members from the UNAVCO WInSAR archive. See the `SSARA.ipynb` notebook for downloading instructions.

# Prepare directories

Importing some python modules and setting up some variables: 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from tqdm import tqdm
import urllib.request
import boto3                      # For talking to s3 bucket
from botocore import UNSIGNED # NEW Change
from botocore.client import Config # NEW Change

#
ASF_USER = ""
ASF_PASS = ""

# the working directory:
home_dir = os.path.join(os.getenv("HOME"), "testing")
PROCESS_DIR = os.path.join(home_dir, "Baja")
work_dir = os.path.join(os.getenv("HOME"), "work")
DATA_DIR =  os.path.join(work_dir, "data/Baja")



In [ ]:
def configure_inputs(outDir): 

    """Write Configuration files for ISCE2 stripmapApp to process Envisat products"""
    cmd_reference_config = '''<component name="reference">
    <property name="IMAGEFILE">
      [/home/jovyan/work/data/Baja/20100328/ASA_IM__0CNPDE20100328_175004_000000162088_00084_42222_9504.N1,
      /home/jovyan/work/data/Baja/20100328/ASA_IM__0CNPDE20100328_175019_000000162088_00084_42222_9504.N1]
        <!-- image files -->
    </property>
    <property name="INSTRUMENT_DIRECTORY">
        /home/jovyan/testing/Baja
    </property>
    <property name="ORBIT_DIRECTORY">
        /home/jovyan/testing/Baja
    </property>
    <property name="OUTPUT">
        reference
    </property>
</component>
'''

    print("writing reference.xml")
    with open(os.path.join(outDir,"reference.xml"), "w") as fid:
        fid.write(cmd_reference_config)
    
    cmd_secondary_config = '''<component name="envi">
    <property name="IMAGEFILE">
      [/home/jovyan/work/data/Baja/20100502/ASA_IM__0CNPDE20100502_175001_000000172089_00084_42723_0354.N1,
      /home/jovyan/work/data/Baja/20100502/ASA_IM__0CNPDE20100502_175016_000000172089_00084_42723_0354.N1] 
        <!-- image files -->
    </property>
    <property name="INSTRUMENT_DIRECTORY">
        /home/jovyan/testing/Baja
    </property>
    <property name="ORBIT_DIRECTORY">
        /home/jovyan/testing/Baja
    </property>
    <property name="OUTPUT">
        secondary
    </property>
  </component>
'''
    
    print("writing secondary.xml")
    with open(os.path.join(outDir,"secondary.xml"), "w") as fid:
        fid.write(cmd_secondary_config)

    
    cmd_stripmap_config = '''<?xml version="1.0" encoding="UTF-8"?>
<insarApp>
  <component name="insar">
    <property name="sensor name">EnviSAT</property>
    <component name="reference">
        <catalog>reference_EnviSAT.xml</catalog>
    </component>
    <component name="secondary">
        <catalog>secondary_EnviSAT.xml</catalog>
    </component>
<!--    <property name="demFilename">
        <value>/Users/fattahi/process/test_roiApp/Alos_Maule_T116/demLat_S39_S35_Lon_W074_W071.dem.wgs84</value>
    </property>
  -->
    <!--
      <property name="do rubbersheetingAzimuth">True</property>
      <property name="do rubbersheetingRange">False</property>
    -->
    <property name="do denseoffsets">True</property>
    <property name="do split spectrum">False</property>
    <property name="unwrapper name">snaphu</property>
    <property name="do dispersive">False</property>
    <property name="dispersive filter kernel x-size">800</property>
    <property name="dispersive filter kernel y-size">800</property>
    <property name="dispersive filter kernel sigma_x">100</property>
    <property name="dispersive filter kernel sigma_y">100</property>
    <property name="dispersive filter kernel rotation">0</property>
    <property name="dispersive filter number of iterations">5</property>
    <property name="dispersive filter mask type">connected_components</property>
    <property name="dispersive filter coherence threshold">0.6</property>

  </component>
</insarApp>'''

    print("writing stripmapApp.xml")
    with open(os.path.join(outDir,"stripmapApp.xml"), "w") as fid:
        fid.write(cmd_stripmap_config)

Check if the PROCESS_DIR and DATA_DIR already exist. If they don't exist, we create them:

In [ ]:

if not os.path.exists(PROCESS_DIR):
    print("create ", PROCESS_DIR)
    os.makedirs(PROCESS_DIR)
else:
    print(PROCESS_DIR, " already exists!")

if not os.path.exists(DATA_DIR):
    print("create ", DATA_DIR)
    os.makedirs(DATA_DIR)
else:
    print(DATA_DIR, " already exists!")

os.chdir(DATA_DIR)

In this tutorial we will process two ESA Envisat acquisitions over Baja California. The two acquisitions cover the 2010 El Mayor-Cucapah Earthquake.


Download the Envisat frames from the UNAVCO WInSAR servers with the SSARA command as in the `SSARA.ipynb` notebook that also explains how to move them to the `$HOME/work/data/Baja` directory.


In [ ]:
#Alternative ssara command
#!ssara_federated_query.py --platform=ALOS --intersectsWith='POLYGON((-155.3 19.5, -155.3 19.8,-155.0 19.8,-155.0 19.5, -155.3 19.5 ))' --print --kml --flightDirection=D --beamMode=FBS,FBD --relativeOrbit=598   -s 2011-01-17 -e 2011-03-07 --download

In the `SSARA.ipynb` notebook you should have created two directories for the two acquisition dates and moved the Envisat files there:

In [ ]:
ls 

In [ ]:
ls 20100328

When you download Envisat data from a data provider, each frame comprises a single image data file that starts with `ASA_IM__0` and ends with `.N1`. In this case, we downloaded two frames for each date.  

Envisat SAR data files have both metadata and binary data in the same file. The metadata at the beginning is ASCII text and can be viewed in your terminal, but the binary data later in the file is not directly viewable (and viewing it may confuse your terminal). ISCE  supports processing the raw (Level 0) or SLC (Level 1) data from Envisat ASAR acquired in imaging or stripmap mode, so make sure the filename starts with `ASA_IM__0` for raw or `ASA_IMS_1` for SLC if it has the standard ESA name.


The acquisition date of the Envisat data is near the beginning of the file name.

In [ ]:
ls 20100502

Now that we have the data ready let's cd to the main PROCESS directroy

In [ ]:
os.chdir(PROCESS_DIR)

To make sure where we are, run pwd:

In [ ]:
!pwd

# Setting up input xml files for processing with stripmapApp

Create the input configuration files (refernce.xml, secondary.xml, stripmapApp.xml) to configure the inputs and the processing parameters.
The configurations files can be created using your favorit editor or by calling the "configure" funstion which is defined at the top of this notebook:


In [ ]:
configure_inputs(PROCESS_DIR)

Here is an example refernce.xml file for this tutorial:

### reference.xml

```xml
<component name="reference">
    <property name="IMAGEFILE">
      [/home/jovyan/work/data/Baja/20100328/ASA_IM__0CNPDE20100328_175004_000000162088_00084_42222_9504.N1,
      /home/jovyan/work/data/Baja/20100328/ASA_IM__0CNPDE20100328_175019_000000162088_00084_42222_9504.N1]
        <!-- image files -->
    </property>
    <property name="INSTRUMENT_DIRECTORY">
        /home/jovyan/testing/Baja
    </property>
    <property name="ORBIT_DIRECTORY">
        /home/jovyan/testing/Baja
    </property>
    <property name="OUTPUT">
        reference
    </property>
</component>
```

### secondary.xml

```xml
<component>
<component name="envi">
    <property name="IMAGEFILE">
      [/home/jovyan/work/data/Baja/20100502/ASA_IM__0CNPDE20100502_175001_000000172089_00084_42723_0354.N1,
      /home/jovyan/work/data/Baja/20100502/ASA_IM__0CNPDE20100502_175016_000000172089_00084_42723_0354.N1] 
        <!-- image files -->
    </property>
    <property name="INSTRUMENT_DIRECTORY">
        /home/jovyan/testing/Baja
    </property>
    <property name="ORBIT_DIRECTORY">
        /home/jovyan/testing/Baja
    </property>
    <property name="OUTPUT">
        secondary
    </property>
  </component>
```

### stripmapApp.xml



```xml
<?xml version="1.0" encoding="UTF-8"?>
<stripmapApp>
  <component name="insar">
    <property name="sensor name">EnviSAT</property>
    <component name="reference">
        <catalog>reference_EnviSAT.xml</catalog>
    </component>
    <component name="secondary">
        <catalog>secondary_EnviSAT.xml</catalog>
    </component>
<!--    <property name="demFilename">
        <value>/Users/fattahi/process/test_roiApp/Alos_Maule_T116/demLat_S39_S35_Lon_W074_W071.dem.wgs84</value>
    </property>
  -->
    <!--
      <property name="do rubbersheetingAzimuth">True</property>
      <property name="do rubbersheetingRange">False</property>
    -->
    <property name="do denseoffsets">True</property>
    <property name="do split spectrum">False</property>
    <property name="unwrapper name">snaphu</property>
    <property name="do dispersive">False</property>
    <property name="dispersive filter kernel x-size">800</property>
    <property name="dispersive filter kernel y-size">800</property>
    <property name="dispersive filter kernel sigma_x">100</property>
    <property name="dispersive filter kernel sigma_y">100</property>
    <property name="dispersive filter kernel rotation">0</property>
    <property name="dispersive filter number of iterations">5</property>
    <property name="dispersive filter mask type">connected_components</property>
    <property name="dispersive filter coherence threshold">0.6</property>

  </component>
</strimapApp>

```

<br>
<div class="alert alert-info">
<b>Note :</b> 

demFilename is commented out in the stripmapApp.xml. This means that user has not specified the DEM. Therefore, isce looks online and download the SRTM dem.

</div>


In the same directory as this `stripmapApp-Baja-Env.ipynb` notebook (`Geo-SInC/EarthScope2024/1.4_Stripmap_Data_Processing_Interferometry/Baja/`, there should be another file called `Baja-Envisat.zip` that contains the Envisat orbit files and instrument description file that are also needed for Envisat processing. You should unzip this file and move those files to the `PROCESS_DIR` (/home/jovyan/testing/Baja).

After downloading the data to process, and setting up the input xml files, we are ready to start processing with stripmapApp. To see a full list of the processing steps run the following command:

In [ ]:
!stripmapApp.py --help --steps

# stripmapApp processing steps

The default setting of this App includes the following steps to generate a geocoded interferogram from raw data or SLC images:

'startup', 'preprocess', 
'formslc',
'verifyDEM', 
'topo', 
'geo2rdr', 
'coarse_resample', 
'misregistration', 
'refined_resample', 
'interferogram', 
'filter', 
'unwrap', 
'geocode'

<br>
<div class="alert alert-info">
<b>Note (to process the interferogram with one command):</b> 

stripmapApp.py stripmapApp.xml --start=startup --end=endup

</div>

However in this tutorial we process the interferogram step by step.

<br>
<div class="alert alert-info">
<b>At the end of each step, you will see a mesage showing the remaining steps:</b> 

The remaining steps are (in order):  [.....]
</div>


### pre-processing

In [ ]:
!stripmapApp.py stripmapApp.xml --start=startup --end=preprocess

By the end of "preprocess", the following folders are created:

reference_raw

secondary_raw

If you look into one of these folders:

In [ ]:
ls reference_raw

`reference.raw` contains the raw data (I/Q real and imaginary parts of each pulse, sampled along track (azimuth direction) with Pulse Repitition Frequency (PRF) and across track(range direction) with Range Sampling Frequency. stripmapApp currently only handles data acquired (or resampled) to a constant PRF. 

### crop raw data

In [ ]:
!stripmapApp.py stripmapApp.xml --start=cropraw --end=cropraw

The "cropraw" step would crop the raw data based on the region of interest if it was requested in the stripmapApp.xml. The region of interest can be added to stripmapApp.xml as:
```xml
<property name="regionOfInterest">[19.0, 19.9, -155.4, -154.7]</property>
```

Since we have not sopecified the region of interest, then "cropraw" will be ignored and the whole frame will be processed.

### focusing 

In [ ]:
!stripmapApp.py stripmapApp.xml --start=formslc --end=formslc

By the end of "formslc", the raw data for both reference and secondary images are focused to SLC images.

In [ ]:
ls reference_slc

reference.slc: Single Look Comlex image for 20110119 acquisition. 

reference.slc.vrt: A gdal VRT file which contains the size, data type, etc.

reference.slc.xml: ISCE xml metadat file

In order to see the number of lines and pixels for an SLC image (or any data readable by GDAL):

In [ ]:
!gdalinfo reference_slc/reference.slc

Display a subset of SLC's amplitude and phase

In [ ]:

ds = gdal.Open("reference_slc/reference.slc", gdal.GA_ReadOnly)
# extract a part of the SLC to display
x0 = 0
y0 = 10000
x_offset = 5000
y_offset = 10000
slc = ds.GetRasterBand(1).ReadAsArray(x0, y0, x_offset, y_offset)
ds = None

fig = plt.figure(figsize=(14, 12))

# display amplitude of the slc
ax = fig.add_subplot(1,2,1)
ax.imshow(np.abs(slc), vmin = -2, vmax=2, cmap='gray')
ax.set_title("amplitude")

#display phase of the slc
ax = fig.add_subplot(1,2,2)
ax.imshow(np.angle(slc))
ax.set_title("phase")

plt.show()

slc = None

### crop SLC

In [ ]:
!stripmapApp.py stripmapApp.xml  --start=cropslc --end=cropslc

Similar to crop raw data but for SLC. Since region of interest has not been specified, the whole frame is processed.

### verifyDEM

Checks if the DEM was given in the input xml file. If the DEM is not given, then the app downloads SRTM DEM from the NASA Land Processes DAAC.

In [ ]:
!stripmapApp.py stripmapApp.xml  --start=verifyDEM --end=verifyDEM

This step will check the DEM file specified in the stripmapApp.xml. If no DEM file has been specified, stripmapApp.py will download the DEM on the fly and track the filename for subsequent processing. 

<div class="alert alert-danger">
<b>POTENTIAL ISSUE:</b> 
You did not set your earthdata credentials as detailed in the ISCE installation. Find instructions here: [Step 2 here](https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+cURL+And+Wget)
</div>

<div class="alert alert-danger">
<b>POTENTIAL ISSUE:</b> 
You did set up your earthdata credentials as detailed in the ISCE installation but you have special characters in your password. "Escape" these characters by adding a backslash in front of them. 
</div>

<div class="alert alert-danger">
<b>POTENTIAL ISSUE:</b> 
The DEM download site is down and returns no or partial data tiles.
</div></div>  


### topo (mapping radar coordinates to geo coordinates)

In [ ]:
!stripmapApp.py stripmapApp.xml  --start=topo --end=topo

At this step, based on the SAR acquisition geometry of the reference Image (including Doppler information), platforms trajectory and an existing DEM, each pixel of the reference image is geolocated. The geolocated coordinates will be at the same coordinate system of the platforms state vectors, which are usually given in WGS84 coordinate system. Moreover the incidence angle and heading angles will be computed for each pixel. 

![title](notebook_docs/Topo.png)

Outputs of the step "topo" are written to "geometry" directory:


In [ ]:
!ls geometry

lat.rdr.full: latitude of each pixel on the ground. "full" stands for full SAR image resolution grid (before multi-looking)

lon.rdr.full: longitude

z.rdr.full: height

los.rdr.full: incidence angle and heading angle

In [ ]:
# Read a bounding box of latitude
ds = gdal.Open('geometry/lat.rdr.full', gdal.GA_ReadOnly)
lat = ds.GetRasterBand(1).ReadAsArray(0,10000,5000, 5000)
ds = None

# Read a bounding box of longitude
ds = gdal.Open('geometry/lon.rdr.full', gdal.GA_ReadOnly)
lon = ds.GetRasterBand(1).ReadAsArray(0,10000,5000, 5000)
ds = None

# Read a bounding box of height
ds = gdal.Open('geometry/z.rdr.full', gdal.GA_ReadOnly)
hgt = ds.GetRasterBand(1).ReadAsArray(0,10000,5000, 5000)
ds = None

fig = plt.figure(figsize=(18, 16))

ax = fig.add_subplot(1,3,1)
cax=ax.imshow(lat)
ax.set_title("latitude")
ax.set_axis_off()
cbar = fig.colorbar(cax, orientation='horizontal')


ax = fig.add_subplot(1,3,2)
cax=ax.imshow(lon)
ax.set_title("longitude")
ax.set_axis_off()
cbar = fig.colorbar(cax, orientation='horizontal')


ax = fig.add_subplot(1,3,3)
cax=ax.imshow(hgt, vmin = -100, vmax=1000)
ax.set_title("height")
ax.set_axis_off()
cbar = fig.colorbar(cax, orientation='horizontal')

plt.show()

lat = None
lon = None
hgt = None

### geo2rdr (mapping from geo coordinates to radar coordinates)

In [ ]:
!stripmapApp.py stripmapApp.xml --start=geo2rdr --end=geo2rdr

At this step, given the geo-ccordinates of each pixel in the reference image (outputs of topo), the range and azimuth time (radar coordinates) is computed given the acquisition geometry and orbit information of the secondary image.  

![title](notebook_docs/Geo2rdr.png)

The computed range and azimuth time for the secondary image, gives the pure geometrical offset, required for resampling the secondary image to the reference image in the next step.

![title](notebook_docs/deltaR.png)

After running this step, the geometrical offsets are available in "offsets" folder:

In [ ]:
!ls offsets

azimuth.off: contains the offsets between reference and secondary images in azimuth direction

range.off:   contains the offsets between reference and secondary images in range direction

In [ ]:
from osgeo import gdal
import matplotlib.pyplot as plt

ds = gdal.Open('offsets/azimuth.off', gdal.GA_ReadOnly)
# extract only part of the data to display
az_offsets = ds.GetRasterBand(1).ReadAsArray(100,100,5000,5000)
ds = None

ds = gdal.Open('offsets/range.off', gdal.GA_ReadOnly)
# extract only part of the data to display
rng_offsets = ds.GetRasterBand(1).ReadAsArray(100,100,5000,5000)
ds = None

fig = plt.figure(figsize=(14, 12))

ax = fig.add_subplot(1,2,1)
cax=ax.imshow(az_offsets)
ax.set_title("azimuth offsets")
ax.set_axis_off()
cbar = fig.colorbar(cax, orientation='horizontal')

ax = fig.add_subplot(1,2,2)
cax = ax.imshow(rng_offsets)
ax.set_title("range offsets")
ax.set_axis_off()
cbar = fig.colorbar(cax, orientation='horizontal')

plt.show()

az_offsets = None
rng_offsets = None

### resampling (using only geometrical offsets)

In [ ]:
!stripmapApp.py stripmapApp.xml --start=coarse_resample --end=coarse_resample

At this step, the gemetrical offsets are used to resample the secondary image to the same grid as the reference image, i.e., the secondary image is co-registered to the reference image. The output of this step is written to "coregisteredSlc" folder.

In [ ]:
!ls coregisteredSlc/

coarse_coreg.slc: is the secondary SLC coregistered to the reference image

In [ ]:
from osgeo import gdal

ds = gdal.Open("coregisteredSlc/coarse_coreg.slc", gdal.GA_ReadOnly)
slc = ds.GetRasterBand(1).ReadAsArray(0, 10000, 5000, 10000)
ds = None

fig = plt.figure()
ax = fig.add_subplot(1,2,1)
ax.imshow(np.abs(slc), vmin = -2, vmax=2, cmap='gray')
ax.set_title("amplitude")

slc = None

### misregistration (estimating constant offsets in range and azimuth directions)

In [ ]:
!stripmapApp.py stripmapApp.xml --start=misregistration --end=misregistration

The range and azimuth offsets derived from pure geometry can be potentially affected by inaccuracy of orbit information or inaccurate DEMs, or inaccurate SAR metadata. The current available DEMs (e.g., SRTM DEMs) are accurate enough to estimate offsets with accuracies of 1/100 of a pixel. The Orbit information of most modern SAR sensors are also precise enough to obtain the same order of accuracy. However, inaccurate metadata (such as timing error, constant range bias), or range bulk delay may affect the estimated offsets. To account for such sources of errors the misregistration step is performed to estimate possible constant offsets between coarse coregistered SLC and reference SLC. For this purpose an incoherent cross correlation is performed. 

The results of the "misregistration" step is written to the "misreg" folder.


In [ ]:
!ls misreg/

In order to extract the estimated misregistration offsets:

In [ ]:
import isce
import isceobj.StripmapProc.StripmapProc as St

stObj=St()
stObj.configure()

az = stObj.loadProduct("misreg/misreg_az.xml")
rng = stObj.loadProduct("misreg/misreg_rg.xml")

print("azimuth misregistration: ", az._coeffs)
print("range misregistration: ", rng._coeffs)

### refine_resample (resampling using geometrical offsets + misregistration)

In [ ]:
!stripmapApp.py stripmapApp.xml --start=refined_resample --end=refined_resample

At this step resampling is re-run to account for the misregistration estimated at the previous step. The new coregisterd SLC (named refined_coreg.slc) is written to the "coregisteredSlc" folder.

In [ ]:
!ls coregisteredSlc/

### optional steps ('dense_offsets', 'rubber_sheet', 'fine_resample', 'split_range_spectrum' , 'sub_band_resample')

In [ ]:
!stripmapApp.py stripmapApp.xml --start=dense_offsets --end=sub_band_resample

These steps are optional and will be skipped if user does not request them in the input xml file. We will get back to these steps in a different session where we estimate ionospheric phase. I have enabled the dense_offsets in the example xml file (`<property name="do denseoffsets">True</property>`) for this earthquake because the large displacements are relevant for pixel offsets. 

### interferogram

In [ ]:
!stripmapApp.py stripmapApp.xml --start=interferogram --end=interferogram

At this step the referene=ce image and refined_coreg.slc is used to generate the interferogram. The generated interferogram is multi-looked based on the user inputs in the input xml file. If user does not specify the number of looks in range and azimuth directions, then they will be estimated based on posting. The default posting is 30 m which can be also specified in the input xml file.

The results of the interferogram step is written to the "interferogram" folder:

In [ ]:
!ls interferogram/

topophase.flat: flattened (geometrical phase removed) and multi-looked interferogram.(one band complex64 data).

topophase.cor: coherence and magnitude for the flattened multi-looked interferogram. (two bands float32 data).

topophase.cor.full: similar to topophase.cor but at full SAR resolution.

topophase.amp: amplitudes of reference amd secondary images. (two bands float32) 


### sub-band interferogram

In [ ]:
!stripmapApp.py stripmapApp.xml --start=sub_band_interferogram --end=sub_band_interferogram

This step will be skipped as we have not asked for ionospheric phase estimation. We will get back to this step in the ionospheric phase estimation notebook.

### filter

In [ ]:
!stripmapApp.py stripmapApp.xml --start=filter --end=filter

A power spectral filter is applied to the multi-looked interferogram to reduce noise.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# reading the multi-looked wrapped interferogram
ds = gdal.Open("interferogram/topophase.flat", gdal.GA_ReadOnly)
igram = ds.GetRasterBand(1).ReadAsArray()
ds = None

# reading the multi-looked un-wrapped interferogram
ds = gdal.Open("interferogram/filt_topophase.flat", gdal.GA_ReadOnly)
filt_igram = ds.GetRasterBand(1).ReadAsArray()
ds = None

fig = plt.figure(figsize=(18, 16))

ax = fig.add_subplot(1,3,1)
ax.imshow(np.abs(igram), vmin = 0 , vmax = 60.0, cmap = 'gray')
ax.set_title("magnitude")
ax.set_axis_off()

ax = fig.add_subplot(1,3,2)
ax.imshow(np.angle(igram), cmap='jet')
ax.plot([1000,2800,2800,1000,1000],[3000,3000,2000,2000,3000],'-k')
ax.set_title("multi-looked interferometric phase")
ax.set_axis_off()

ax = fig.add_subplot(1,3,3)
ax.imshow(np.angle(filt_igram), cmap='jet')
ax.plot([1000,2800,2800,1000,1000],[3000,3000,2000,2000,3000],'-k')
ax.set_title("multi-looked & filtered phase")
#ax.set_axis_off()

fig = plt.figure(figsize=(18, 16))

ax = fig.add_subplot(1,3,1)
ax.imshow(np.abs(igram[2000:3000, 1000:2800]), vmin = 0 , vmax = 60.0, cmap = 'gray')
ax.set_title("magnitude")
ax.set_axis_off()

ax = fig.add_subplot(1,3,2)
ax.imshow(np.angle(igram[2000:3000, 1000:2800]), cmap='jet')
ax.plot([600,1200,1200,600,600],[600,600,100,100,600],'--k')
ax.set_title("multi-looked interferometric phase")
ax.set_axis_off()

ax = fig.add_subplot(1,3,3)
ax.imshow(np.angle(filt_igram[2000:3000, 1000:2800]), cmap='jet')
ax.plot([600,1200,1200,600,600],[600,600,100,100,600],'--k')
ax.set_title("multi-looked & filtered phase")
ax.set_axis_off()

fig = plt.figure(figsize=(18, 16))

ax = fig.add_subplot(1,3,1)
ax.imshow(np.abs(igram[2100:2600, 1600:2200]), vmin = 0 , vmax = 60.0, cmap = 'gray')
ax.set_title("magnitude")
ax.set_axis_off()

ax = fig.add_subplot(1,3,2)
ax.imshow(np.angle(igram[2100:2600, 1600:2200]), cmap='jet')
ax.set_title("multi-looked interferometric phase")
ax.set_axis_off()

ax = fig.add_subplot(1,3,3)
ax.imshow(np.angle(filt_igram[2100:2600, 1600:2200]), cmap='jet')
ax.set_title("multi-looked & filtered phase")
ax.set_axis_off()

filt_igram = None
igram = None


<br>
<div class="alert alert-info">
<b>Note :</b> 
The interferometric phase, shows ground displacement caused by dike opening event in March 2011, along the east rift zone of Kīlauea Volcano, Hawaii.

</div>


### optional steps ('filter_low_band', 'filter_high_band')

In [ ]:
!stripmapApp.py stripmapApp.xml --start=filter_low_band  --end=filter_high_band

These steps will be skipped since we have not asked for ionospheric phase estimation in the input xml file.

### unwrap

In [ ]:
!stripmapApp.py stripmapApp.xml --start=unwrap  --end=unwrap

At this step the wrapped phase of the filtered and multi-looked interferogram is unwrapped. The unwrapped interferogram is a two band data with magnitude and phase components.

In [ ]:
import matplotlib.pyplot as plt

# reading the multi-looked wrapped interferogram
ds = gdal.Open("interferogram/filt_topophase.flat", gdal.GA_ReadOnly)
igram = ds.GetRasterBand(1).ReadAsArray()
ds = None

# reading the multi-looked unwrapped interferogram
ds = gdal.Open("interferogram/filt_topophase.unw", gdal.GA_ReadOnly)
igram_unw = ds.GetRasterBand(2).ReadAsArray()
ds = None

# reading the connected component file
ds = gdal.Open("interferogram/filt_topophase.unw.conncomp", gdal.GA_ReadOnly)
connected_components = ds.GetRasterBand(1).ReadAsArray()
ds = None


fig = plt.figure(figsize=(18, 16))

ax = fig.add_subplot(1,3,1)
cax=ax.imshow(np.angle(igram), cmap='jet')
ax.set_title("wrapped")
#ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[-3.14,0,3.14],orientation='horizontal')
cbar.ax.set_xticklabels(["$-\pi$",0,"$\pi$"])

ax = fig.add_subplot(1,3,2)
cax = ax.imshow(igram_unw, vmin = -50 , vmax = 50.0, cmap = 'jet')
ax.set_title("unwrapped")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[-50,0, 50], orientation='horizontal')


ax = fig.add_subplot(1,3,3)
cax = ax.imshow(connected_components, cmap = 'jet')
ax.set_title("components")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[0, 1] , orientation='horizontal')
cbar.ax.set_xticklabels([0,1])


connected_components = None

<br>
<div class="alert alert-info">
<b>Note (wrapped vs unwrapped) :</b> 
Note the colorscale for the wrapped and unwrapped interferograms. The wrapped interferometric phase varies from $-\pi$ to $\pi$, while the unwrapped interferogram varies from -15 to 15 radians.
</div>

<br>
<div class="alert alert-info">
<b>Note :</b> 
The connected components file is a product of the phase unwrapping. Each interferogram may have several connected compoenets. The unwrapped phase within each component is expected to be correctly unwrapped. However, there might be $2\pi$ phase jumps between the components. Advanced ISCE users may use the 2-stage unwrapping to adjust ambiguities among different components. stripmapApp currently does not support 2-stage unwrapping. Look for this option in future releases.  
</div>


In [ ]:

profile_wrapped_1 = np.angle(igram[2400,1500:1650])
profile_unwrapped_1 = igram_unw[2400,1500:1650]
profile_wrapped_2 = np.angle(igram[2400,1500:2000])
profile_unwrapped_2 = igram_unw[2400,1500:2000]

fig = plt.figure(figsize=(20,8))

ax = fig.add_subplot(2,3,1)
cax=ax.plot(profile_wrapped_1)
ax.set_title("wrapped")

ax = fig.add_subplot(2,3,2)
cax=ax.plot(profile_unwrapped_1)
ax.set_title("unwrapped")

ax = fig.add_subplot(2,3,3)
cax=ax.plot(np.round((profile_unwrapped_1-profile_wrapped_1)/2.0/np.pi))
ax.set_title("(unwrapped - wrapped)/(2$\pi$)")

ax = fig.add_subplot(2,3,4)
cax=ax.plot(profile_wrapped_2)
ax.set_title("wrapped")

ax = fig.add_subplot(2,3,5)
cax=ax.plot(profile_unwrapped_2)
ax.set_title("unwrapped")

ax = fig.add_subplot(2,3,6)
cax=ax.plot((profile_unwrapped_2-profile_wrapped_2)/2.0/np.pi)
ax.set_title("(unwrapped - wrapped)/(2$\pi$)")


igram = None
igram_unw = None

### optional steps ('unwrap_low_band', 'unwrap_high_band', 'ionosphere')

In [ ]:
!stripmapApp.py stripmapApp.xml --start=unwrap_low_band  --end=ionosphere

Since we have not asked for ionospheric phase estimation, all these steps will be skipped.

### geocoding

In [ ]:
!stripmapApp.py stripmapApp.xml --start=geocode  --end=geocode

## Final Geocoded Unwrapped phase
Now we can plot the geocoded unwrapped phase that is sometimes called a GUNW product. We will work with the GRFN ARIA-processed Sentinel-1 GUNW products later in the course.

In [ ]:
# reading the multi-looked wrapped interferogram
ds = gdal.Open("interferogram/filt_topophase.unw.geo", gdal.GA_ReadOnly)
unw_geocoded = ds.GetRasterBand(2).ReadAsArray()
ds = None


fig = plt.figure(figsize=(14,12))

ax = fig.add_subplot(1,1,1)
cax = ax.imshow(unw_geocoded, vmin = -15 , vmax = 15.0, cmap = 'jet')
ax.set_title("geocoded unwrapped")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[-15,0, 15], orientation='horizontal')

plt.show()
unw_geocoded = None

## Geocoded dense offsets

We can also geocode the dense offsets, which is an optional step

In [ ]:
!stripmapApp.py stripmapApp.xml --start=geocodeoffsets 


The output file of the dense offsets calculation run earlier, `dense_offsets.bil`, is a two-band file containing the azimuth and range offsets in radar coordinates, placed in the `denseOffsets` folder . The correlation coefficient is contained in the `dense_offsets_snr.bil` file.

Here is a plot of the azimuth and range offsets:


In [ ]:
# This will only work if you enabled pixel offset generation
# reading the dense offsets in radar coordinates
ds = gdal.Open("denseOffsets/denseOffsets.bil", gdal.GA_ReadOnly)
range_off = ds.GetRasterBand(2).ReadAsArray()
azim_off = ds.GetRasterBand(1).ReadAsArray()
ds = None


fig = plt.figure(figsize=(14,12))

ax = fig.add_subplot(1,2,1)
cax = ax.imshow(range_off, vmin = -0.2 , vmax = 0.2, cmap = 'jet')
ax.set_title("DENSE RANGE OFFSETS [pixels]")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[-0.2,0, 0.2], orientation='horizontal')

ax = fig.add_subplot(1,2,2)
cax = ax.imshow(azim_off, vmin = -0.5 , vmax = 0.5, cmap = 'jet')
ax.set_title("DENSE AZIMUTH OFFSETS [pixels]")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[-0.5,0, 0.5], orientation='horizontal')

plt.show()
range_off = None
azim_off = None

We can now look at the geocoded offsets. Because this earthquake had a large amount of strike-slip displacement, so there is a significant signal in the azimuth or along-track offsets. See the cells at the bottom of this notebook where we calculate the range and azimuth pixel spacing of this data.

In [ ]:
# reading the geocoded dense offsets 
ds = gdal.Open("denseOffsets/denseOffsets.bil.geo", gdal.GA_ReadOnly)
range_off = ds.GetRasterBand(2).ReadAsArray()
azim_off = ds.GetRasterBand(1).ReadAsArray()
ds = None


fig = plt.figure(figsize=(14,12))

ax = fig.add_subplot(1,2,1)
cax = ax.imshow(range_off, vmin = -0.1 , vmax = 0.1, cmap = 'jet')
ax.set_title("DENSE RANGE OFFSETS [pixels]")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[-0.1,0, 0.1], orientation='horizontal')

ax = fig.add_subplot(1,2,2)
cax = ax.imshow(azim_off, vmin = -0.5 , vmax = 0.5, cmap = 'jet')
ax.set_title("DENSE AZIMUTH OFFSETS [pixels]")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[-0.5,0, 0.5], orientation='horizontal')

plt.show()
range_off = None
azim_off = None

# Supplementary Information

### understanding xml files

The format of this type of file may seem unfamiliar or strange to you, but with the following description of the basics of the format, it will hopefully become more familiar.   The first thing to point out is that the indentations and line breaks seen above are not required and are simply used to make the structure more clear and the file more readable to humans.  The xml file provides structure to data for consumption by a computer.  As far as the computer is concerned the data structure is equally readable if all of the information were contained on a single very long line, but human readers would have a hard time reading it in that format. 

The next thing to point out is the method by which the data are structured through the use of tags and attributes.  An item enclosed in the < (less-than) and > (greater-than) symbols is referred to as a tag.  The name enclosed in the < and > symbols is the name of the tag.  Every tag in an xml file must have an associated closing tag that contains the same name but starts with the symbol </ and ends with the symbol >.  This is the basic unit of structure given to the data.  Data are enclosed inside of opening and closing tags that have names identifying the enclosed data.  This structure is nested to any order of nesting necessary to represent the data.  The Python language (in which the ISCE user interface is written) provides powerful tools to parse the xml structure into a data structure object and to very easily “walk” through the structure of that object.  

In the above xml file the first and last tags in the file are a tag pair: <stripmapApp> and </stripmapApp> (note again, tags must come in pairs like this).  The first of these two tags, or the opening tag,  marks the beginning of the contents of the tag and the second of these two tags, or the closing tag, marks the end of the contents of the tag.  ISCE expects a “file tag” of this nature to bracket all inputs contained in the file.  The actual name of the file tag, as far as ISCE is concerned, is user selectable.  In this example it is used, as a convenience to the user, to document the ISCE application, named insarApp.py, for which it is meant to provide inputs; it could have been named <foo> and insarApp.py would have been equally happy provided that the closing tag were </foo>.  

The next tag  is <component name="insar">.  Its closing tag </component> is located at the penultimate line of the file (one line above the </insar> tag). The name of this tag is component and it has an attribute called name with value “insarApp”. The component tags bound a collection of information that is used by a computational element within ISCE that has the name specified by the name attribute.   The name “insarApp”  in the first component tag tells ISCE that the enclosed information correspond to a functional component in ISCE named “insarApp”, which in this case is actually the application that is run at the command line.  

In general, component tags contain information in the form of other component tags or property tags, all of which can be nested to any required level. In this example the  insarApp component contains a property tag and two other component tags.

The first tag we see in the insarApp component tag is the property tag with attribute name=“sensor name”. The property tag contains a value tag that contains the name of the sensor, ALOS in this case.  The next tag is a component tag with attribute name=”reference”. This tag contains a catalog tag containing  reference.xml.  The catalog tag in general informs ISCE to look in the named file (reference.xml in this case) for the contents of the current tag.  The next component tag has the same structure with the catalog  tag containing a different file named secondary.xml.

### Extra configuration parameters 

The input configuration file in this tutorial only included mandatory parameters including the reference and secondary images, which are enough to run the application. This means that the application is configured with default parameters hardwired in the code or computed during processing. 
For custom processing, user may want to set parameters in the input configuration file. In the following a few more parameters are shown that can be added to stripmapApp.xml. 

### regionOfInterest

To specify a region of interest to process:

```xml
<property name="regionOfInterest">[South, North, West, East]</property>
```

Example: 

```xml
<property name="regionOfInterest">[19.0, 19.9, -155.4, -154.7]</property>
```

Default: Full frame is processed.

### range looks
number of looks in range direction 

```xml
<property name="range looks">USER_INPUT</property>
```

Deafult: is computed based on the posting parameter.

### azimuth looks
number of looks in azimuth direction 

Deafult: is computed based on the posting parameter.


### posting
Interferogram posting in meters.

```xml
<property name="posting">USER_INPUT</property>
```

Default: 30

<br>
<div class="alert alert-info">
<b>Note :</b> 
If "range looks" and "azimuth looks" have not been specified, then posting is used to compute them such that the interferogram is generated with a roughly square pixels size with each dimension close to the "posting" parameter.

</div>

### filter strength

strength of the adaptive filter used for filtering the wrapped interferogram

```xml
<property name="filter strength">USER_INPUT</property>
```

Default: 0.5


### useHighResolutionDemOnly
```xml
<property name="useHighResolutionDemOnly">True</property>
```

If True and a dem is not specified in input, it will only
    download the SRTM highest resolution dem if it is available
    and fill the missing portion with null values (typically -32767)

Default: False

### do unwrap

To turn phase unwrapping off
```xml
<property name="do unwrap">False</property>
```

Default: True


### unwrapper name
To choose the name of the phase unwrapping method. e.g., to choose "snaphu" for phase unwrapping
```xml
<property name="unwrapper name">snaphu</property>
```

Default: "icu".




### do rubbersheeting

To turn on rubbersheeting (estimating azimuth offsets caused by strong ionospheric scentilation)

```xml
<property name="do rubbersheeting">True</property>
```
Default : False

### rubber sheet SNR Threshold

```xml
<property name="rubber sheet SNR Threshold">USER_INPUT</property>
```
If "do rubbersheeting" is turned on, then this values is used to mask out azimuth offsets with SNR less that the input threshold. 

Default: 5

### rubber sheet filter size
the size of the median filter used for filtering the azimuth offsets

```xml
<property name="rubber sheet filter size">USER_INPUT</property>
```

Default: 8

### do denseoffsets
turn on the dense offsets computation from cross correlation

```xml
<property name="do denseoffsets">True</property>
```

Default: False

<br>
<div class="alert alert-info">
<b>Note :</b> 

If "do rubbersheeting" is turned on, then dense offsets computation is turned on regardless of the user input for "do denseoffsets"

</div>

### setting the dense offsets parameters 

```xml
<property name="dense window width">USER_INPUT</property>
<property name="dense window height">USER_INPUT</property>
<property name="dense search width">USER_INPUT</property>
<property name="dense search height">USER_INPUT</property>
<property name="dense skip width">USER_INPUT</property>
<property name="dense skip height">USER_INPUT</property>
```

Default values:
<br>
    dense window width  = 64 
<br>
    dense window height = 64
<br>
    dense search width  = 20
<br>
    dense search height = 20
<br>
    dense skip width    = 32
<br>
    dense skip height   = 32


### geocode list

List of products to be geocoded.
```xml
<property name="geocode list">"a list of files to geocode">
```
Default: multilooked, filtered wrapped and unwrapped interferograms, coherence, ionospehric phase

### offset geocode list
List of offset-specific files to geocode
```xml
<property name="offset geocode list">"a list of offset files to geocode">
```


### do split spectrum

turn on split spectrum 

```xml
<property name="do split spectrum">True</property>
```

Default: False

### do dispersive
turn on disperive phase estimation

```xml
<property name="do dispersive">True</property>
```

Default: False

<br>
<div class="alert alert-info">
<b>Note :</b> 

By turning on "do dispersive", the user input for "do split spectrum" is ignored and the split spectrum will be turned on as it is needed for dispersive phase estimation. 

</div>


### control the filter kernel for filtering the dispersive phase  
```xml
<property name="dispersive filter kernel x-size">800</property>
<property name="dispersive filter kernel y-size">800</property>
<property name="dispersive filter kernel sigma_x">100</property>
<property name="dispersive filter kernel sigma_y">100</property>
<property name="dispersive filter kernel rotation">0</property>
<property name="dispersive filter number of iterations">5</property>
<property name="dispersive filter mask type">coherence</property>
<property name="dispersive filter coherence threshold">0.6</property>
    
```    



### processing data from other stripmap sensors

stripmapApp.py is able to process the stripmap data from the following sensors. So far it has been sucessfully tested on the following sensors: 
    <br>
- ALOS1 (Raw and SLC)
-  ALOS2 (SLC, one frame)
-  COSMO_SkyMed (Raw and SLC)
-  ERS
-  ENVISAT (Raw and SLC)
-   Radarsat-1
-  Radarsat-2
-  TerraSARX
-  TanDEMX (or PAZ)
-  Sentinel1 (stripmap mode)
    
        
    
### Sample input data xml for different sensors:

#### Envisat: 
```xml

<component name="reference">
    <property name="IMAGEFILE">data/ASA_IMS_1PNESA20050519_140259_000000172037_00239_16826_0000.N1</property>
    <property name="INSTRUMENT_DIRECTORY">/u/k-data/agram/sat_metadata/ENV/INS_DIR</property>
    <property name="ORBIT_DIRECTORY">/u/k-data/agram/sat_metadata/ENV/Doris/VOR</property>
    <property name="OUTPUT">
        20050519
    </property>
</component>
The above example is for Envisat SLC (ASA_IMS_1) images. StripmapApp also supports Envisat Raw (ASA_IM__0) images.

```

<br>
<div class="alert alert-info">
<b>Note :</b> 
Note that a directory that contains the orbits and another with the Instrument calibration files is required for processing ENVISAT data. 
</div>


### Sentinel-1 stripmap:
```xml
    <component name="reference">
      <property name="orbit directory">/u/data/sat_metadata/S1/aux_poeorb/</property>
      <property name="output">20151024</property>
      <property name="safe">/u/data/S1A_S1_SLC__1SSV_20151024T234201_20151024T234230_008301_00BB43_068C.zip</property>
    </component>
    <component name="secondary">
      <property name="orbit directory">/u/k-raw/sat_metadata/S1/aux_poeorb/</property>
      <property name="output">20150930</property>
      <property name="safe">/u/data/S1A_S1_SLC__1SSV_20150930T234200_20150930T234230_007951_00B1CC_121C.zip</property>
    </component>
```

<br>
<div class="alert alert-info">
<b>Note :</b> 
Note that for processing the Sentinel-1 data a directory that contains the orbits is required. 
</div>

### ALOS2 SLC
```xml
<component>
    <property name="IMAGEFILE">
        <value>data/20141114/ALOS2025732920-141114/IMG-HH-ALOS2025732920-141114-UBSL1.1__D</value>
    </property>
    <property name="LEADERFILE">
        <value>data/20141114/ALOS2025732920-141114/LED-ALOS2025732920-141114-UBSL1.1__D</value>
    </property>
    <property name="OUTPUT">
        <value>20141114</value>
    </property>
</component>
```

### ALOS1 raw data
``` xml
<component>
    <property name="IMAGEFILE">
       <value>[data/20110119/ALPSRP265743230-L1.0/IMG-HH-ALPSRP265743230-H1.0__D]</value>
    </property>
    <property name="LEADERFILE">
        <value>[data/20110119/ALPSRP265743230-L1.0/LED-ALPSRP265743230-H1.0__D]</value>
    </property>
    <property name="OUTPUT">
        <value>20110119</value>
    </property>
</component>
```

### CosmoSkyMed raw or SLC data

```xml
<component name="reference">
    <property name="HDF5">data/CSKS3_RAW_B_HI_03_HH_RD_SF_20111007021527_20111007021534.h5</property>
    <property name="OUTPUT">
        20111007
    </property>
</component>

```

### TerraSAR-X and TanDEM-X

```xml
<component name="reference">
    <property name="xml">PATH_TO_TSX_DATA_XML</property>
    <property name="OUTPUT">OUTPUT_NAME</property> 
</component>    
```


### Using ISCE as a python library

ISCE can be used a python library. Users can develop their own workflows within ISCE framework. Here are few simple examples where we try to call isce modules:


#### Example 1: (extract metadata, range and azimuth pixel size)

In [ ]:
import isce
import isceobj
import isceobj.StripmapProc.StripmapProc as St
from isceobj.Planet.Planet import Planet

stObj = St()
stObj.configure()
frame = stObj.loadProduct("reference_slc.xml")
print("Wavelength = {0} m".format(frame.radarWavelegth))
print("Slant Range Pixel Size = {0} m".format(frame.instrument.rangePixelSize))

#For azimuth pixel size we need to multiply azimuth time interval by the platform velocity along the track

# the acquisition time at the middle of the scene
t_mid = frame.sensingMid

#get the orbit for t_mid
st_mid=frame.orbit.interpolateOrbit(t_mid)

# platform velocity
Vs = st_mid.getScalarVelocity()

# pulse repitition frequency
prf = frame.instrument.PRF

#Azimuth time interval 
ATI = 1.0/prf

#Azimuth Pixel size
az_pixel_size = ATI*Vs
print("Azimuth Pixel Size = {0} m".format(az_pixel_size))





#### Example 2: compute ground range pixels size

In [ ]:
r0 = frame.startingRange
rmax = frame.getFarRange()
rng =(r0+rmax)/2

elp = Planet(pname='Earth').ellipsoid
tmid = frame.sensingMid

sv = frame.orbit.interpolateOrbit( tmid, method='hermite') #.getPosition()
llh = elp.xyz_to_llh(sv.getPosition())


hdg = frame.orbit.getENUHeading(tmid)
elp.setSCH(llh[0], llh[1], hdg)
sch, vsch = elp.xyzdot_to_schdot(sv.getPosition(), sv.getVelocity())

Re = elp.pegRadCur
H = sch[2]
cos_beta_e = (Re**2 + (Re + H)**2 -rng**2)/(2*Re*(Re+H))
sin_bet_e = np.sqrt(1 - cos_beta_e**2)
sin_theta_i = sin_bet_e*(Re + H)/rng
print("incidence angle at the middle of the swath: ", np.arcsin(sin_theta_i)*180.0/np.pi)
groundRangeRes = frame.instrument.rangePixelSize/sin_theta_i
print("Ground range pixel size: {0} m ".format(groundRangeRes))


<br>
<div class="alert alert-info">
<b>Note :</b> 
One can easily get the incidence angle from the los.rdr file inside geometry folder. Even without opening the file, here is a way to get the statistics and the average value of the incidence angle:  gdalinfo geometry/los.rdr -stats
</div>